In [1]:
import pandas as pd
import os

# === ETAPA 1: EXTRAÇÃO ===
def carregar_dados(caminho):
    tabelas = {
        os.path.splitext(arquivo)[0]: pd.read_csv(os.path.join(caminho, arquivo), encoding='latin1')
        for arquivo in os.listdir(caminho)
        if arquivo.endswith('.csv')
    }
    return tabelas

**Objetivo**:
Carregar automaticamente todos os arquivos .csv de um diretório específico, convertendo cada um em um DataFrame e armazenando-os em um dicionário nomeado conforme o arquivo.

**Detalhes técnicos**:
1. **os.listdir()** lista os arquivos do diretório.
2. **pd.read_csv()** lê cada CSV.
3. **encoding='latin1'** garante a leitura correta de acentos e caracteres especiais.

In [2]:
# === ETAPA 2: TRANSFORMAÇÃO ===
def limpar_colunas(tabelas):
    for df in tabelas.values():
        df.columns = df.columns.str.strip()
    return tabelas

def transformar_dados(tabelas):
    orders = tabelas['orders']
    channels = tabelas['channels']
    deliveries = tabelas['deliveries']
    drivers = tabelas['drivers']
    hubs = tabelas['hubs']
    payments = tabelas['payments']
    stores = tabelas['stores']

    df = (
        orders
        .merge(channels, on='channel_id', how='left')
        .merge(stores, on='store_id', how='left')
        .merge(payments, left_on='order_id', right_on='payment_order_id', how='left')
        .merge(deliveries, on='delivery_order_id', how='left')
        .merge(drivers, on='driver_id', how='left')
        .merge(hubs, on='hub_id', how='left')
    )
    return df


**Objetivo**:
Remover espaços em branco dos nomes das colunas para padronização e evitar erros nas junções.

**Detalhes técnicos**:
1. **limpar_colunas**: Remove espaços extras nos nomes das colunas, garantindo consistência na hora de acessar ou unir os dados.
2. **transformar_dados**: Realiza junções (merge) entre as principais tabelas do negócio, como pedidos, canais, entregas, motoristas, etc. O resultado é um DataFrame consolidado que representa uma visão integrada de cada pedido.

In [3]:
# === ETAPA 2.1: TRATAMENTOS AVANÇADOS ===

def ajustar_tipos(df):
    colunas_data = [col for col in df.columns if 'date' in col.lower() or 'data' in col.lower()]
    for col in colunas_data:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype(str).str.strip().str.lower()
    
    return df

def tratar_nulos(df):
    for col in df.select_dtypes(include='number').columns:
        df[col].fillna(df[col].median(), inplace=True)

    for col in df.select_dtypes(include='object').columns:
        df[col].fillna('desconhecido', inplace=True)
    
    return df

def remover_duplicatas(df):
    return df.drop_duplicates()

def padronizar_texto(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.strip().str.lower().replace({'-': ' ', '_': ' '}, regex=True)
    return df

def remover_colunas_constantes(df):
    return df.loc[:, df.nunique() > 1]

def tratar_outliers_iqr(df):
    for col in df.select_dtypes(include='number').columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        df = df[(df[col] >= limite_inferior) & (df[col] <= limite_superior)]
    return df


**Objetivo**:
Essa etapa refina a base de dados para garantir que ela esteja limpa, padronizada e confiável. Isso reduz ruídos e viabiliza análises mais precisas, além de evitar que erros ou distorções afetem o desempenho de modelos e relatórios.

**Detalhes técnicos**:
**ajustar_tipos**: Converte colunas de datas e padroniza textos para minúsculo e sem espaços.
**tratar_nulos**: Preenche valores ausentes com a mediana (numéricos) ou "desconhecido" (textuais).
**remover_duplicatas**: Elimina registros repetidos.
**padronizar_texto**: Remove caracteres inconsistentes e padroniza nomes.
**remover_colunas_constantes**: Exclui colunas sem variação, que não agregam valor.
**tratar_outliers_iqr**: Remove valores fora do intervalo interquartil (IQR), diminuindo o impacto de outliers.

In [4]:
# === ETAPA 3: CARGA ===
def salvar_dados(df, caminho_saida):
    df.to_csv(caminho_saida, index=False)

# === EXECUÇÃO DO ETL ===
if __name__ == '__main__':
    caminho_entrada = '../data/raw'
    caminho_saida = '../data/processed/dataset_final.csv'

    tabelas = carregar_dados(caminho_entrada)
    tabelas = limpar_colunas(tabelas)
    df_final = transformar_dados(tabelas)
    salvar_dados(df_final, caminho_saida)

    print("✅ ETL finalizado com sucesso!")

✅ ETL finalizado com sucesso!


**Objetivo**
Salva o DataFrame final processado em um novo arquivo .csv, pronto para uso posterior.